# Notebook para probar cosas

### Librerias que seran utilizadas

* sklearn
* tensorflow
* keras
* smart-open
* matplotlib
* seaborn
* numpy

### Pasos a seguir

* practicar tutorial del profe
* definir forma de acceder a los datos  (**LISTO**)
* definir una primera forma de obtener los descriptores del texto   (**LISTO**) -> usando countVectorizer
* definir una segunda forma de obtener los descriptores del texto
* definir pre procesamiento de los datos (ordenamiento, limpieza, escalado, etc)  (**LISTO**)
* definir un regresor para obtener los descriptores de la imagen en base a descriptores de texto (**LISTO**)
* entrenar utilizando pocas neuronas escondidas para no demorar tanto y testear resultados
* realizar validaciones sobre la red, verificar sobre ajuste y metricas disponibles, iterar cuanto sea necesario.
* entrenar para la red mas grande.
* definir el calculador de distancias y la mertrica que se utilizara para realizar las busquedas
* realizar pruebas sobre los resultados y crear graficos
* concluir y generar un informe bonito

### Bibliografia

* http://queirozf.com/entries/evaluation-metrics-for-ranking-problems-introduction-and-examples
* https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-regression-ceee5a9eadff
* https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
* https://datascienceplus.com/keras-regression-based-neural-networks/
* tutorial del profe

vayamos agregando los avances aqui, cosa que cuando uno quiera avanzar algo el otro pueda usar su avance y asi.

Todos los archivos python que agregues metelos en /src, las funciones de un archivo se importan asi:

    from src.foo import function
    
en /develop meti el codigo de MLP en tensorflow y del java del profe, creo que no los necesitaremos pero los deje por si acaso. Cualquier cosa que creas que podriamos utilizar pero no estes seguro agregala a develop

### accesso a los datos

In [34]:
# probemos el codigo del profe para leer datos
# cargando descriptores de imagenes
import numpy
import os

def load_file(file_names, file_vectors, num_vectors, vector_dimensions):
    assert os.path.isfile(file_names), "no existe archivo " + file_names
    assert os.path.isfile(file_vectors), "no existe archivo " + file_vectors
    print("leyendo " + file_names)
    names = [line.strip() for line in open(file_names)]
    assert num_vectors == len(names), "no cuadra largo archivo " + len(names)
    print("leyendo " + file_vectors)
    mat = numpy.fromfile(file_vectors, dtype=numpy.float32)
    vectors = numpy.reshape(mat, (num_vectors, vector_dimensions))
    print(str(num_vectors) + " vectores de largo " + str(vector_dimensions))
    return (names, vectors)

def load_train_vectors():
    return load_file("./data/train_data/train_images_names.txt", "./data/train_data/train_images_vectors.bin", 20000, 2048)
    
def load_test_vectors():
    return load_file("test_A_images_names.txt", "test_A_images_vectors.bin", 1000, 2048)

(train_names, train_vectors) = load_train_vectors()
# (test_names, test_vectors) = load_test_vectors()
print(np.shape(train_names), np.shape(train_vectors))
print("Imagen \"" + train_names[0] + "\" tiene descriptor visual " + str(train_vectors[0]) + " de dimension " + str(len(train_vectors[0])))

leyendo ./data/train_data/train_images_names.txt
leyendo ./data/train_data/train_images_vectors.bin
20000 vectores de largo 2048
(20000,) (20000, 2048)
Imagen "COCO_train2014_000000000086.jpg" tiene descriptor visual [0.402616   0.14888027 0.09194826 ... 0.08761896 1.0989506  0.31788927] de dimension 2048


In [35]:
# cargar captions de train
def load_captions(file_captions):
    assert os.path.isfile(file_captions), "no existe archivo " + file_captions
    return [line.strip().split("\t") for line in open(file_captions, encoding='utf-8')]

# test_captions = load_file("test_A_captions.txt")
train_captions = load_captions("./data/train_data/train_captions.txt")
print(np.shape(train_captions))
for i in range(6):
    print("Imagen \"" + train_captions[i][0] + "\" tiene caption \"" + train_captions[i][1] + "\"")

(100000, 2)
Imagen "COCO_train2014_000000000086.jpg" tiene caption "Un hombre en un una vieja bicicleta de moda en el bosque"
Imagen "COCO_train2014_000000000086.jpg" tiene caption "Un hombre montado en una bicicleta de motor a través de un bosque."
Imagen "COCO_train2014_000000000086.jpg" tiene caption "Un hombre sentado en una motocicleta en el bosque."
Imagen "COCO_train2014_000000000086.jpg" tiene caption "Una persona que mira hacia abajo en algo mientras está sentado en una bicicleta."
Imagen "COCO_train2014_000000000086.jpg" tiene caption "Una persona joven está en una vieja bicicleta muy adornado."
Imagen "COCO_train2014_000000000077.jpg" tiene caption "un grupo de adolescentes saltando una rampa en sus monopatines"


* tengo 20000 vectores de largo 2048, es decir la matriz de entrenamiento y sera de shape (20000, 2048)
* pero tengo 5 captions por cada imaganes y por tanto, 5 captions por cada descriptor de imagen, entonces mis shapes seran:

    - (100000, $\alpha$) para la matriz de entrenamiento de entrada (X)
    - (100000, 2048) para la matriz de entrenamiento de resultados (y)

* donde $\alpha$ esta dado por el descriptor de texto que creemos. El profe sugirio que seria de largo 1000

### obtener descriptores de texto

* countVectorizer usando ngram_range (1, 1) es igual a un unigrama, que es directamente el metodo de bag of words
* tf-idf: aun no se lo que es ...
* word embedding: es posible implementar esto dentro de la red, pero requiere usar convoluciones dentro de la regresion y para ello se necesita GPU y tal vez agregue mucha complejidad.

ahora usare solo bag of words (countVectorizing)

In [56]:
# obtengo solo los captions (asumo que los textos vienen ordenados)
train_captions = np.array(train_captions)
train_texts = train_captions[:,1]

In [58]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=False,ngram_range=(1,1),max_df=0.8,min_df=0.01,binary=True)
vectorizer.fit(train_texts)

X_train = vectorizer.transform(train_texts)
# X_test  = vectorizer.transform(texts_test)

print("Entrenamiento: {}".format(X_train.shape))
# print("Test: {}".format(X_test.shape))
X_train

Entrenamiento: (100000, 153)


<100000x153 sparse matrix of type '<class 'numpy.int64'>'
	with 717616 stored elements in Compressed Sparse Row format>

* ordenemos un poco todo

In [87]:
def load_file(file_names, file_vectors, num_vectors, vector_dimensions):
    assert os.path.isfile(file_names), "no existe archivo " + file_names
    assert os.path.isfile(file_vectors), "no existe archivo " + file_vectors
    print("leyendo " + file_names)
    names = [line.strip() for line in open(file_names)]
    assert num_vectors == len(names), "no cuadra largo archivo " + len(names)
    print("leyendo " + file_vectors)
    mat = numpy.fromfile(file_vectors, dtype=numpy.float32)
    vectors = numpy.reshape(mat, (num_vectors, vector_dimensions))
    print(str(num_vectors) + " vectores de largo " + str(vector_dimensions))
    return (names, vectors)
    
def load_captions(file_captions):
    assert os.path.isfile(file_captions), "no existe archivo " + file_captions
    return [line.strip().split("\t") for line in open(file_captions, encoding='utf-8')]

# crearemos una clase que simplifique todo el proceso de adquirir los datos
class DataHandler(object):
    def __init__(self, root_folder="./"):
        self.root = root_folder
        self.vectors_dimension = 2048
        self._data = {}
        
    def _load(self, names_file, vectors_file, captions_file, num_vectors):
        (names, vectors) = load_file(names_file, vectors_file, num_vectors, self.vectors_dimension)
        captions = load_captions(captions_file)
        vectors_for_captions = []
        for vector in vectors:
            for _ in range(5):
                vectors_for_captions.append(vector)
        return names, np.array(vectors_for_captions), np.array(captions)
        

    def load_train(self, folder=""):
        if len(folder) > 0 and folder[0] != '/':
            folder = '/' + folder
        names_file = "{}{}/train_images_names.txt".format(self.root, folder)
        captions_file = "{}{}/train_captions.txt".format(self.root, folder)
        vectors_file = "{}{}/train_images_vectors.bin".format(self.root, folder)
        return self._load(names_file, vectors_file, captions_file, 20000)
    
    def load_test(self, folder=""):
        if len(folder) > 0 and folder[0] != '/':
            folder = '/' + folder
        names_file = "{}{}/test_A_images_names.txt".format(self.root, folder)
        captions_file = "{}{}/test_A_captions.txt".format(self.root, folder)
        vectors_file = "{}{}/test_A_images_vectors.bin".format(self.root, folder)
        return self._load(names_file, vectors_file, captions_file, 1000)
    
    def get_data_count_vectorizer(self, train_folder="/train_data", test_folder="/test_A_data", **kwargs):
        train_names, train_vectors, train_image_captions = self.load_train(folder=train_folder)
        test_names, test_vectors, test_image_captions = self.load_test(folder=test_folder)
        
        print("doing vectorization with CountVectorizer")
        train_captions = train_image_captions[:,1]
        test_captions = test_image_captions[:,1]
        vectorizer = CountVectorizer(lowercase=False,ngram_range=(1,1),max_df=0.8,min_df=0.01,binary=True)
        print("fitting bag of words ...", end="")
        vectorizer.fit(train_captions)
        print("done")
        
        print("getting vectors transforms ...", end="")
        train_text_descriptors = vectorizer.transform(train_captions)
        test_text_descriptors = vectorizer.transform(test_captions)
        print("done")
        
        # esto podria ser util
#         self._data.update({"train_names": train_names, 
#                            "train_text_descriptors": train_text_descriptors,
#                            "train_image_descriptors": train_vectors,
#                            "test_names": test_names, 
#                            "test_text_descriptors": test_text_descriptors, 
#                            "test_image_descriptors": test_vectors})

        return train_text_descriptors, test_text_descriptors, train_vectors, test_vectors
    

In [88]:
# luego bastaria con hacer
data = DataHandler(root_folder="./data")
X_train, X_test, y_train, y_test = data.get_data_count_vectorizer()
print("shapes")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

leyendo ./data/train_data/train_images_names.txt
leyendo ./data/train_data/train_images_vectors.bin
20000 vectores de largo 2048
leyendo ./data/test_A_data/test_A_images_names.txt
leyendo ./data/test_A_data/test_A_images_vectors.bin
1000 vectores de largo 2048
doing vectorization with CountVectorizer
fitting bag of words ...done
getting vectors transforms ...done
shapes
X_train: (100000, 153)
y_train: (100000, 2048)
X_test: (5000, 153)
y_test: (5000, 2048)


### Regresor con keras

In [102]:
from keras.models import Sequential
from keras.layers import Dense

class Regressor(object):
    
    def __init__(self, input_dim, output_dim, n_layers=2, layer_size=2048):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self._n = n_layers
        self._size = layer_size
        self.optimizer = "adam"
        self.loss = "mean_squared_error"
        self.metrics = ["mae", "accuracy"]
        self.model = None
        self._regressor = None
        
    def build(self):
        self.model = Sequential()
        self.model.add(Dense(self._size, input_dim=self.input_dim, kernel_initializer='normal', activation='relu'))
        i = self._n - 1
        while i > 0:
            self.model.add(Dense(self._size, activation='relu'))
            i -= 1
        self.model.add(Dense(self.output_dim, activation='linear'))
        self.model.summary()
        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)
        
    def fit(self, X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, shuffle=True):
        results=self.model.fit(X_train,y_train, epochs=epochs, 
                               validation_split=validation_split,shuffle=shuffle,
                               batch_size=batch_size)
        return results
    
    def predict(self, X):
        # aqui probablemente debamos normalizar X
        return self.model.predict(X)
        # y probablemente debamos des-normalizar X e y_predicho


In [104]:
# esto se debiese llamar simplemente asi: (ejemplo con un regresor pequeño)
reg = Regressor(X_train.shape[1], y_train.shape[1], n_layers=1, layer_size=500)
reg.build()
reg.fit(X_train, y_train, epochs=10)
# y luego predecir algun nuevo dato (se va a usar con los datos de test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 500)               77000     
_________________________________________________________________
dense_12 (Dense)             (None, 2048)              1026048   
Total params: 1,103,048
Trainable params: 1,103,048
Non-trainable params: 0
_________________________________________________________________
Train on 90000 samples, validate on 10000 samples
Epoch 1/10
90000/90000 [==============================] - 36s 397us/step - loss: 0.5854 - mean_absolute_error: 0.4675 - acc: 0.1579 - val_loss: 0.5817 - val_mean_absolute_error: 0.4710 - val_acc: 0.1808
Epoch 2/10
90000/90000 [==============================] - 35s 394us/step - loss: 0.5641 - mean_absolute_error: 0.4590 - acc: 0.1802 - val_loss: 0.5774 - val_mean_absolute_error: 0.4630 - val_acc: 0.1875
Epoch 3/10
90000/90000 [==============================] - 35s 389us/step

In [17]:
from sklearn.preprocessing import  MinMaxScaler
sc= MinMaxScaler()
X= sc.fit_transform(X)
y= y.reshape(-1,1)
y=sc.fit_transform(y)